In [1]:
pip install pandas scikit-learn tensorflow numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install tensorflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:

#  Load data 
X = pd.read_csv("C:/LINCS_DATA/project/features/pair_feature_matrix.csv").values
y = pd.read_csv("C:/LINCS_DATA/project/features/pair_labels.csv")['label'].values

In [37]:
# ⚙️ Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [38]:
# 📉 Apply PCA (Reduce dimensions to 300)
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_scaled)

In [39]:
# 🔀 Split into Train + Temp (Val + Test)
X_train, X_temp, y_train, y_temp = train_test_split(X_pca, y, test_size=0.3, random_state=42, stratify=y)

# 🔁 Split Temp into Validation + Test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)



In [40]:

# 🧪 Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [41]:
# 🧠 Build the MLP model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_smote.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\Reem Ramadan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

In [43]:
# 🏋️ Train the model
history = model.fit(
    X_train_smote, y_train_smote,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.4843 - loss: 0.9149 - val_accuracy: 0.4384 - val_loss: 1.3092
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6966 - loss: 0.6063 - val_accuracy: 0.5890 - val_loss: 0.8363
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7170 - loss: 0.5408 - val_accuracy: 0.6438 - val_loss: 0.7577
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7563 - loss: 0.5028 - val_accuracy: 0.6575 - val_loss: 0.7115
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7565 - loss: 0.4805 - val_accuracy: 0.6575 - val_loss: 0.7445
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7740 - loss: 0.4666 - val_accuracy: 0.6164 - val_loss: 0.8286
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8136 - loss: 0.4256 - val_accuracy: 0.6712 - val_loss: 0.7509
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8456 - loss: 0.3518 - val_accuracy: 0.

In [44]:
# 📊 Evaluate on Test Set
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [45]:
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, classification_report

# Metrics
print("✅ Evaluation on Test Set:")
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

✅ Evaluation on Test Set:
F1 Score: 0.5901639344262295
ROC-AUC Score: 0.6873469387755102

Confusion Matrix:
[[31 18]
 [ 7 18]]

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.63      0.71        49
           1       0.50      0.72      0.59        25

    accuracy                           0.66        74
   macro avg       0.66      0.68      0.65        74
weighted avg       0.71      0.66      0.67        74

